In [1]:
import pnet_loader
import util
import torch
import seaborn as sns
import pandas as pd
import numpy as np
import Pnet
import random
import pickle
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
import os

%load_ext autoreload
%autoreload 2

Generate small test dataset

In [2]:
# rna_ext_val = pd.read_csv('/mnt/disks/pancan/data/mel_dfci_2019/data_RNA_Seq_expression_tpm_all_sample_Zscores.txt',
#                           delimiter='\t').set_index('Hugo_Symbol').T.drop('Entrez_Gene_Id').dropna(axis=1)
# cna_ext_val = pd.read_csv('/mnt/disks/pancan/data/mel_dfci_2019/data_CNA.txt',
#                           delimiter='\t').set_index('Hugo_Symbol').T.dropna(axis=1)
# ext_val = pd.read_csv('/mnt/disks/pancan/data/mel_dfci_2019/data_clinical_sample.txt',
#                              delimiter='\t').set_index('Sample Identifier').iloc[4:]
# important_genes = list(pd.read_csv('/mnt/disks/pancan/m1000/cancer_genes.txt')['genes'].values)
# joint_genes = list(set(important_genes).intersection(list(rna_ext_val.columns), list(cna_ext_val.columns)))
# gene_list = random.sample(joint_genes, 500)
# random_genes_a = list(rna_ext_val.sample(5, axis=1).columns)
# random_genes_b = list(cna_ext_val.sample(5, axis=1).columns)
# joint_samples = list(rna_ext_val.sample(20).join(cna_ext_val, rsuffix='_cna', how='inner').index)
# random_samples_a = list(rna_ext_val.sample(5, axis=0).index)
# random_samples_b = list(cna_ext_val.sample(5, axis=0).index)
# random_samples_c = list(cna_ext_val.sample(5, axis=0).index)
# random_samples_d = list(cna_ext_val.sample(5, axis=0).index)
# test_rna = rna_ext_val.loc[joint_samples+random_samples_a][joint_genes+random_genes_a].copy().drop_duplicates()
# test_cna = cna_ext_val.loc[joint_samples+random_samples_b][joint_genes+random_genes_b].copy().drop_duplicates()
# test_add = ext_val.loc[joint_samples+random_samples_c][['Purity', 'Ploidy']].copy().drop_duplicates()
# test_y = ext_val.loc[joint_samples+random_samples_d][['Heterogeneity']].copy().drop_duplicates()
# test_rna.reset_index(inplace=True)
# test_cna.reset_index(inplace=True)
# test_add.reset_index(inplace=True)
# test_y.reset_index(inplace=True)
# test_rna.rename(columns={'index': 'sample_id'}, inplace=True)
# test_cna.rename(columns={'index': 'sample_id'}, inplace=True)
# test_add.rename(columns={'Sample Identifier': 'sample_id'}, inplace=True)
# test_y.rename(columns={'Sample Identifier': 'sample_id'}, inplace=True)
# test_rna.to_csv('../data/test_data/rna.csv', index=False)
# test_cna.to_csv('../data/test_data/cna.csv', index=False)
# test_add.to_csv('../data/test_data/add.csv', index=False)
# test_y.to_csv('../data/test_data/y.csv', index=False)
# with open('../data/test_data/gene_sublist.txt', 'wb') as fp:
#     pickle.dump(gene_list, fp)

Read test data

In [3]:
test_rna = pd.read_csv('../data/test_data/rna.csv').set_index('sample_id')
test_cna = pd.read_csv('../data/test_data/cna.csv').set_index('sample_id')
test_add = pd.read_csv('../data/test_data/add.csv').set_index('sample_id')
test_y = pd.read_csv('../data/test_data/y.csv').set_index('sample_id')

with open('../data/test_data/gene_sublist.txt', 'rb') as fp:
    gene_list = pickle.load(fp)

In [4]:
genetic_data = {'rna': test_rna, 'cna': test_cna}

In [5]:
train_dataset, test_dataset = pnet_loader.generate_train_test(genetic_data,
                                                              test_y, 
                                                              additional_data=test_add,
                                                              test_split=0.2,
                                                              gene_set=gene_list,
                                                              collinear_features=2)

Given 2 Input modalities
Found 20 overlapping indicies
Initializing Train Dataset
Found 500 overlapping genes
generated input DataFrame of size (16, 1000)
Initializing Test Dataset
Found 500 overlapping genes
generated input DataFrame of size (4, 1000)
Replace input of: EBF1_cna with collinear feature.
Replace input of: USP6 with collinear feature.


In [6]:
assert set(gene_list) == set(train_dataset.genes), 'Training dataset expected to have the same gene set as in file'
assert train_dataset.genes == list(train_dataset.input_df.columns)[:500], 'Training data genes should be ordered \
                                                                            as stored in the genes variable'
assert train_dataset.input_df.shape == torch.Size([16, 1000]), 'Input DataFrame expected to be a of size\
                                                        [16, 1000], got: {}'.format(train_dataset.input_df.shape)
assert train_dataset.x.shape == torch.Size([16, 1000]), 'Small train dataset expected to be a tensor of size\
                                                        [16, 1000], got: {}'.format(train_dataset.x.shape)
assert train_dataset.y.shape == torch.Size([16, 1]), 'Small train dataset expected to be a tensor of size\
                                                        [16, 1], got: {}'.format(train_dataset.y.shape)


In [7]:
import ReactomeNetwork

In [8]:
reactome_network = ReactomeNetwork.ReactomeNetwork(train_dataset.get_genes())

Found 500 overlapping genes


In [9]:
train_loader, val_loader = pnet_loader.to_dataloader(train_dataset, test_dataset, 64)

In [51]:
test_y_bin = test_y.apply(lambda x: round(2.3*x))
test_y_bin

,Heterogeneity
sample_id,
Sample159,1.0
Sample146,0.0
Sample117,1.0
Sample10,1.0
Sample61,1.0
Sample17,0.0
Sample24,0.0
Sample78,1.0
Sample45,1.0


In [52]:
model, train_scores, test_scores, train_dataset, test_dataset = Pnet.run(genetic_data,
                                                                         test_y_bin,
                                                                         verbose=True,
                                                                         early_stopping=True)

Given 2 Input modalities
Found 20 overlapping indicies
Initializing Train Dataset
Found 649 overlapping genes
generated input DataFrame of size (16, 1298)
Initializing Test Dataset
Found 649 overlapping genes
generated input DataFrame of size (4, 1298)
Found 649 overlapping genes
We are sending to cuda
BCEWithLogitsLoss()
BCEWithLogitsLoss()
Epoch 1 of 300
Train Loss: 1.7392191886901855
Test Loss: 0.6940104961395264
BCEWithLogitsLoss()
BCEWithLogitsLoss()
Epoch 2 of 300
Train Loss: 1.737900733947754
Test Loss: 0.6959637403488159
BCEWithLogitsLoss()
BCEWithLogitsLoss()
Epoch 3 of 300
Train Loss: 1.6664104461669922
Test Loss: 0.6979390382766724
BCEWithLogitsLoss()
BCEWithLogitsLoss()
Epoch 4 of 300
Train Loss: 1.6824779510498047
Test Loss: 0.6996645331382751
BCEWithLogitsLoss()
BCEWithLogitsLoss()
Epoch 5 of 300
Train Loss: 1.6153452396392822
Test Loss: 0.7014715075492859
exceeded delta
BCEWithLogitsLoss()
BCEWithLogitsLoss()
Epoch 6 of 300
Train Loss: 1.5708953142166138
Test Loss: 0.703

In [53]:
x_train = train_dataset.x
additional_train = train_dataset.additional
y_train = train_dataset.y
x_test = test_dataset.x
additional_test = test_dataset.additional
y_test = test_dataset.y

In [54]:
model.to('cpu')
pred, preds = model(x_test, additional_test)
y_pred_proba = pred.detach().numpy().squeeze()


In [56]:
pred

tensor([[0.0031],
        [0.0029],
        [0.0032],
        [0.0035]], grad_fn=<AddmmBackward0>)

In [100]:
y_test

tensor([[1.],
        [1.],
        [0.],
        [1.]])

In [101]:
for p in model.attn.parameters():
    print(p)

Parameter containing:
tensor([[ 0.3277, -0.3369,  0.3249,  0.1117,  0.0676]], requires_grad=True)
Parameter containing:
tensor([0.1254], requires_grad=True)


In [49]:
x = torch.randn(1, 3)
x

tensor([[ 1.1193,  0.0246, -0.9385]])

In [44]:
a =torch.randint(10,(10,1))

In [45]:
import torch.nn as nn

In [35]:
m(a[1])

tensor([0.5792, 0.5397])

In [36]:
a[1]

tensor([0.3194, 0.1592])